In [209]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Embedding, LSTM, Input, Activation, Permute, Dropout, add, dot, concatenate
from keras.utils import to_categorical
from pickle import dump, load
from nltk.corpus import words
import nltk
import spacy
import keras

import numpy as np
import pickle
import random
import re

random.seed(42)

In [210]:
nlp = spacy.load('en_core_web_sm')

## Load Dataset

In [211]:
with open('qa6_yes-no-questions_test.txt', 'rb') as f:
    text_test = f.read().decode('utf-8')

with open('qa6_yes-no-questions_train.txt', 'rb') as f:
    text_train = f.read().decode('utf-8')

In [212]:
# with open("train_qa.txt", "rb") as fp:   # Unpickling
#     text_train =  pickle.load(fp)

# with open("test_qa.txt", "rb") as fp:   # Unpickling
#     text_test =  pickle.load(fp)

In [213]:
text_test

'1 Mary got the milk there.\n2 John moved to the bedroom.\n3 Is John in the kitchen? \tno\t2\n4 Mary discarded the milk.\n5 John went to the garden.\n6 Is John in the kitchen? \tno\t5\n7 Daniel moved to the bedroom.\n8 Daniel went to the garden.\n9 Is John in the garden? \tyes\t5\n10 Daniel travelled to the bathroom.\n11 Sandra travelled to the bedroom.\n12 Is Daniel in the bathroom? \tyes\t10\n13 Mary took the football there.\n14 Sandra grabbed the milk there.\n15 Is Daniel in the bedroom? \tno\t10\n1 Daniel went back to the kitchen.\n2 Mary grabbed the apple there.\n3 Is Daniel in the office? \tno\t1\n4 Daniel journeyed to the office.\n5 John went back to the office.\n6 Is Daniel in the hallway? \tno\t4\n7 Mary left the apple.\n8 Daniel went to the hallway.\n9 Is Daniel in the hallway? \tyes\t8\n10 John went to the hallway.\n11 Daniel picked up the milk there.\n12 Is John in the kitchen? \tno\t10\n13 John grabbed the football there.\n14 Mary got the apple there.\n15 Is Daniel in the 

In [214]:
text_train

'1 Mary moved to the bathroom.\n2 Sandra journeyed to the bedroom.\n3 Is Sandra in the hallway? \tno\t2\n4 Mary went back to the bedroom.\n5 Daniel went back to the hallway.\n6 Is Daniel in the bathroom? \tno\t5\n7 Sandra went to the kitchen.\n8 Daniel went back to the bathroom.\n9 Is Daniel in the office? \tno\t8\n10 Daniel picked up the football there.\n11 Daniel went to the bedroom.\n12 Is Daniel in the bedroom? \tyes\t11\n13 John travelled to the office.\n14 Sandra went to the garden.\n15 Is Daniel in the bedroom? \tyes\t11\n1 Sandra got the football there.\n2 Mary went to the bedroom.\n3 Is Mary in the bedroom? \tyes\t2\n4 Daniel got the apple there.\n5 Sandra travelled to the hallway.\n6 Is Sandra in the office? \tno\t5\n7 Sandra moved to the garden.\n8 Mary travelled to the kitchen.\n9 Is Sandra in the bathroom? \tno\t7\n10 Sandra went back to the bedroom.\n11 Daniel put down the apple.\n12 Is Sandra in the bathroom? \tno\t10\n13 Sandra put down the football.\n14 Sandra journeye

## Data Preprocessing

In [215]:
def preprocessing_data(txt):
    data = []
    record = []
    temp = None
    start = True
    for sentence in txt.split('\n'):
        doc = nlp(sentence)
        token = [i.text for i in doc]
        if len(token) == 0:
            break
        if start:
            if token[0] == '1':
                temp = []
            if temp:
                record.append(temp)
                record[0].extend(token[1:])
                temp = []
            else:
                record.append(token[1:])
            start = False
        elif '?' in token:
            record.append(token[1:-4])
            record.append(token[-3])
            temp = record[0].copy()
            data.append(record.copy())
            record = []
            start = True
        else:
            record[0].extend(token[1:])
    return data

In [216]:
text_train = preprocessing_data(text_train)
text_train

[[['Mary',
   'moved',
   'to',
   'the',
   'bathroom',
   '.',
   'Sandra',
   'journeyed',
   'to',
   'the',
   'bedroom',
   '.'],
  ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
  'no'],
 [['Mary',
   'moved',
   'to',
   'the',
   'bathroom',
   '.',
   'Sandra',
   'journeyed',
   'to',
   'the',
   'bedroom',
   '.',
   'Mary',
   'went',
   'back',
   'to',
   'the',
   'bedroom',
   '.',
   'Daniel',
   'went',
   'back',
   'to',
   'the',
   'hallway',
   '.'],
  ['Is', 'Daniel', 'in', 'the', 'bathroom', '?'],
  'no'],
 [['Mary',
   'moved',
   'to',
   'the',
   'bathroom',
   '.',
   'Sandra',
   'journeyed',
   'to',
   'the',
   'bedroom',
   '.',
   'Mary',
   'went',
   'back',
   'to',
   'the',
   'bedroom',
   '.',
   'Daniel',
   'went',
   'back',
   'to',
   'the',
   'hallway',
   '.',
   'Sandra',
   'went',
   'to',
   'the',
   'kitchen',
   '.',
   'Daniel',
   'went',
   'back',
   'to',
   'the',
   'bathroom',
   '.'],
  ['Is', 'Daniel', 'in', 'the', '

In [217]:
text_test = preprocessing_data(text_test)
text_test

[[['Mary',
   'got',
   'the',
   'milk',
   'there',
   '.',
   'John',
   'moved',
   'to',
   'the',
   'bedroom',
   '.'],
  ['Is', 'John', 'in', 'the', 'kitchen', '?'],
  'no'],
 [['Mary',
   'got',
   'the',
   'milk',
   'there',
   '.',
   'John',
   'moved',
   'to',
   'the',
   'bedroom',
   '.',
   'Mary',
   'discarded',
   'the',
   'milk',
   '.',
   'John',
   'went',
   'to',
   'the',
   'garden',
   '.'],
  ['Is', 'John', 'in', 'the', 'kitchen', '?'],
  'no'],
 [['Mary',
   'got',
   'the',
   'milk',
   'there',
   '.',
   'John',
   'moved',
   'to',
   'the',
   'bedroom',
   '.',
   'Mary',
   'discarded',
   'the',
   'milk',
   '.',
   'John',
   'went',
   'to',
   'the',
   'garden',
   '.',
   'Daniel',
   'moved',
   'to',
   'the',
   'bedroom',
   '.',
   'Daniel',
   'went',
   'to',
   'the',
   'garden',
   '.'],
  ['Is', 'John', 'in', 'the', 'garden', '?'],
  'yes'],
 [['Mary',
   'got',
   'the',
   'milk',
   'there',
   '.',
   'John',
   'moved',


## Create a Vocabulary to Store all Words

In [218]:
vocab = set()

In [219]:
merged_data = text_train + text_test

In [220]:
for story, question, _ in merged_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [221]:
vocab.add('yes')
vocab.add('no')
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [222]:
VOCAB_LEN = len(vocab) + 1
VOCAB_LEN

38

In [223]:
MAX_STORY_LEN = max(len(data[0]) for data in merged_data)
MAX_STORY_LEN

156

In [224]:
MAX_QUESTION_LEN = max(len(data[1]) for data in merged_data)
MAX_QUESTION_LEN

6

## Vectorize the Data

In [225]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [226]:
tokenizer.word_index

{'bedroom': 1,
 'picked': 2,
 'travelled': 3,
 'took': 4,
 'grabbed': 5,
 'back': 6,
 'kitchen': 7,
 'yes': 8,
 'in': 9,
 'hallway': 10,
 'bathroom': 11,
 'moved': 12,
 'left': 13,
 'mary': 14,
 'milk': 15,
 'discarded': 16,
 'sandra': 17,
 '?': 18,
 'football': 19,
 'is': 20,
 'office': 21,
 'journeyed': 22,
 'dropped': 23,
 'there': 24,
 'went': 25,
 'to': 26,
 'no': 27,
 'john': 28,
 'down': 29,
 '.': 30,
 'the': 31,
 'got': 32,
 'put': 33,
 'up': 34,
 'daniel': 35,
 'apple': 36,
 'garden': 37}

In [227]:
tokenizer.word_index['yes']

8

In [228]:
tokenizer.word_index['no']

27

In [229]:
# train_story_text = []
# train_question_text = []
# train_answers = []

# for story,question,answer in train_data:
#     train_story_text.append(story)
#     train_question_text.append(question)

# len(train_story_text)

In [230]:
# train_story_seq = tokenizer.texts_to_sequences(train_story_text)
# len(train_story_seq)

In [231]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=MAX_STORY_LEN, max_question_len=MAX_QUESTION_LEN):
    story_vectors = []
    question_vectors = []
    answer_indices = []  # Use indices instead of one-hot vectors

    for story, question, answer in data:
        # Vectorize the story and question
        story_vectors.append(
            [word_index.get(word.lower(), 0) for word in story[-max_story_len:]]
        )
        question_vectors.append(
            [word_index.get(word.lower(), 0) for word in question[-max_question_len:]]
        )

        # Store the answer as an index
        answer_indices.append(word_index.get(answer.lower(), 0))

    # Pad all sequences at once
    story_vectors = pad_sequences(story_vectors, maxlen=max_story_len)
    question_vectors = pad_sequences(question_vectors, maxlen=max_question_len)
    answer_indices = np.array(answer_indices)

    return story_vectors, question_vectors, answer_indices


In [232]:
inputs_train, queries_train, answers_train = vectorize_stories(text_train)

In [233]:
inputs_test, queries_test, answers_test = vectorize_stories(text_test)

In [234]:
inputs_train

array([[ 0,  0,  0, ..., 31,  1, 30],
       [ 0,  0,  0, ..., 31, 10, 30],
       [ 0,  0,  0, ..., 31, 11, 30],
       ...,
       [ 0,  0,  0, ..., 31,  1, 30],
       [ 0,  0,  0, ..., 15, 24, 30],
       [ 0,  0,  0, ..., 36, 24, 30]], dtype=int32)

In [235]:
inputs_test

array([[ 0,  0,  0, ..., 31,  1, 30],
       [ 0,  0,  0, ..., 31, 37, 30],
       [ 0,  0,  0, ..., 31, 37, 30],
       ...,
       [ 0,  0,  0, ..., 31, 36, 30],
       [ 0,  0,  0, ..., 31, 37, 30],
       [ 0,  0,  0, ..., 36, 24, 30]], dtype=int32)

In [236]:
answers_train = to_categorical(answers_train, num_classes=(len(tokenizer.word_index) + 1))
print(answers_train.shape)
answers_train

(10000, 38)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [237]:
answers_test = to_categorical(answers_test, num_classes=VOCAB_LEN)
print(answers_test.shape)
answers_test

(1000, 38)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Create End-to-End Memory Network (MemN2N) Model

![image.png](attachment:image.png)

### Input Placeholders for the Stories and Questions

In [238]:
input_sequence = Input((MAX_STORY_LEN, ))
question = Input((MAX_QUESTION_LEN, ))

### Encoders

In [239]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=VOCAB_LEN,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [240]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=VOCAB_LEN,output_dim=MAX_QUESTION_LEN))
input_encoder_c.add(Dropout(0.3))

In [241]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=VOCAB_LEN, output_dim=64, input_length=MAX_QUESTION_LEN))
question_encoder.add(Dropout(0.3))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


### Encode the Sequences

In [242]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [243]:
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [244]:
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [245]:
answer = concatenate([response, question_encoded])
answer = LSTM(32)(answer)  # (samples, 32)
answer = Dropout(0.5)(answer)
answer = Dense(VOCAB_LEN)(answer)  # (samples, vocab_size)
answer = Activation('softmax')(answer)

In [246]:
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14            │ (None, 156)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_15            │ (None, 6)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_8 (Sequential) │ (None, 156, 64)        │          2,432 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_10             │ (None, 6, 64)          │          2,432 │ input_layer_15[0][0]   │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dot_2 (Dot)               │ (None, 156, 6)         │              0 │ sequential_8[0][0],    │
│                           │                        │                │ sequential_10[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activation) │ (None, 156, 6)         │              0 │ dot_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_9 (Sequential) │ (None, 156, 6)         │            228 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 156, 6)         │              0 │ activation_4[0][0],    │
│                           │                        │                │ sequential_9[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ permute_2 (Permute)       │ (None, 6, 156)         │              0 │ add_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 6, 220)         │              0 │ permute_2[0][0],       │
│ (Concatenate)             │                        │                │ sequential_10[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ (None, 32)             │         32,384 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_11 (Dropout)      │ (None, 32)             │              0 │ lstm_2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 38)             │          1,254 │ dropout_11[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_5 (Activation) │ (None, 38)             │              0 │ dense_4[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 38,730 (151.29 KB)

 Trainable params: 38,730 (151.29 KB)

 Non-trainable params: 0 (0.00 B)

## Train the Model

In [247]:
history = model.fit([inputs_train, queries_train], answers_train,batch_size=16,epochs=150,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.4859 - loss: 1.0625 - val_accuracy: 0.4970 - val_loss: 0.6959
Epoch 2/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.4983 - loss: 0.7003 - val_accuracy: 0.5030 - val_loss: 0.6948
Epoch 3/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.4986 - loss: 0.6965 - val_accuracy: 0.4970 - val_loss: 0.6933
Epoch 4/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.5019 - loss: 0.6954 - val_accuracy: 0.5030 - val_loss: 0.6932
Epoch 5/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.4887 - loss: 0.6964 - val_accuracy: 0.5020 - val_loss: 0.6932
Epoch 6/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.4943 - loss: 0.6963 - val_accuracy: 0.5030 - val_loss: 0.6935
Epoch 7/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.5002 - loss: 0.6957 - val_accuracy: 0.4970 - val_loss: 0.6934
Epoch 8/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5011 - loss: 0.69

## Make Predictions

In [248]:
# model.load_weights('chatbot_120_epochs.h5')

In [249]:
predictions = model.predict(([inputs_test, queries_test]))

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


### Check the Output of a Sample in Testing Data

In [250]:
idx = np.random.randint(1, len(text_test), 5)
for i in idx:
    print("Story:", ' '.join(text_test[i][0]))
    print("Question:", ' '.join(text_test[i][1]))
    print("Answer (True):", text_test[i][2])
    for key, val in tokenizer.word_index.items():
      if val == np.argmax(predictions[i]):
        k = key
    print("Predicted answer is:", k)
    print("Probability of certainty:", predictions[1][np.argmax(predictions[1])])
    print('\n\n')


Story: Daniel went to the office . Sandra travelled to the hallway .
Question: Is Daniel in the office ?
Answer (True): yes
Predicted answer is: yes
Probability of certainty: 0.9999328



Story: John travelled to the office . Daniel went back to the kitchen . Daniel got the milk there . Sandra went to the office . Sandra travelled to the hallway . John went to the bathroom .
Question: Is Sandra in the kitchen ?
Answer (True): no
Predicted answer is: no
Probability of certainty: 0.9999328



Story: Mary went to the office . Daniel journeyed to the hallway . Mary went to the bedroom . Sandra went to the garden . Sandra took the football there . Mary journeyed to the bathroom .
Question: Is Mary in the office ?
Answer (True): no
Predicted answer is: no
Probability of certainty: 0.9999328



Story: Daniel went to the office . Sandra travelled to the hallway . Mary went to the kitchen . John journeyed to the hallway . John got the football there . John discarded the football . Mary travelle

### Testing the Model using Own Data (Story and Question)

In [251]:
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_question = "Is the football in the garden ?"
mydata = [(my_story.split(),my_question.split(),'yes')]

my_story,my_ques,my_ans = vectorize_stories(mydata)

In [252]:
pred_results = model.predict(([ my_story, my_ques]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [253]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  yes
Probability of certainty was:  0.9264953
